In [6]:
import gspread
import pandas as pd
import os
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import argparse
import random, string
import psycopg2
import time
from creds import rds_username, rds_password, rds_url
from creds import rds_port, rds_db

# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('../creds.json', scope)


client = gspread.authorize(creds)
sheet = client.open('jukebox pipeline')
sheet_instance = sheet.get_worksheet(0)
records_data = sheet_instance.get_all_records()
records_df = pd.DataFrame.from_dict(records_data)

In [38]:
conn = psycopg2.connect(
    host=rds_url,
    port=rds_port,
    dbname=rds_db,
    user=rds_username,
    password=rds_password)

def write2database(conn, table, data):
	cur = conn.cursor()
	cur.execute(table, (data))
	conn.commit()
	cur.close()

add_song = """ INSERT INTO afm_songs_prod VALUES (now(), %s, %s, %s, %s, %s, %s, %s, %s)"""

In [39]:
for i,r in records_df.loc[36:].iterrows():
    for j in range(0,6):
        song_id = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(6))
        data4db = ("{}_{}".format(r['run_id'], song_id),r['run_id'],r['genre'],r['artist'],r['audio_file'],r['dna_artist'], r['link'] + "item_{}.wav".format(j), "batch2")
        write2database(conn, add_song, data4db)
        print(data4db)

('vaKXAop2UWjbnMHY_EgJtWQ', 'vaKXAop2UWjbnMHY', 'rock', 'the weeknd', 'jukebox/data/primes/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED.wav', 'nobide', 'http://matlaberp2.media.mit.edu:8000/outputs/5/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED_theweeknd_rock_prompt6_dur25/level_0/item_0.wav', 'batch2')
('vaKXAop2UWjbnMHY_4dyVts', 'vaKXAop2UWjbnMHY', 'rock', 'the weeknd', 'jukebox/data/primes/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED.wav', 'nobide', 'http://matlaberp2.media.mit.edu:8000/outputs/5/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED_theweeknd_rock_prompt6_dur25/level_0/item_1.wav', 'batch2')
('vaKXAop2UWjbnMHY_lCNhVn', 'vaKXAop2UWjbnMHY', 'rock', 'the weeknd', 'jukebox/data/primes/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED.wav', 'nobide', 'http://matlaberp2.media.mit.edu:8000/outputs/5/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED_theweeknd_rock_prompt6_dur25/level_0/item_2.wav', 'batch2')
('vaKXAop2UWjbnMHY_e8skjv', 'vaKXAop2UWjbnMHY', 'rock', 'the weeknd', 'ju

('yFBbbSzNNjDfITJX_j9R9l6', 'yFBbbSzNNjDfITJX', 'folk', 'the weeknd', 'jukebox/data/primes/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED.wav', 'nobide', 'http://matlaberp2.media.mit.edu:8000/outputs/5/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED_theweeknd_folk_prompt6_dur25/level_0/item_3.wav', 'batch2')
('yFBbbSzNNjDfITJX_mcTHkm', 'yFBbbSzNNjDfITJX', 'folk', 'the weeknd', 'jukebox/data/primes/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED.wav', 'nobide', 'http://matlaberp2.media.mit.edu:8000/outputs/5/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED_theweeknd_folk_prompt6_dur25/level_0/item_4.wav', 'batch2')
('yFBbbSzNNjDfITJX_Op8yqy', 'yFBbbSzNNjDfITJX', 'folk', 'the weeknd', 'jukebox/data/primes/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED.wav', 'nobide', 'http://matlaberp2.media.mit.edu:8000/outputs/5/Nobide_CC13_crystalcastles_soul_item_2_TRIMMED_theweeknd_folk_prompt6_dur25/level_0/item_5.wav', 'batch2')


In [40]:
conn.close()